In [1]:
:!pwd

/projects/biproductions


: 1

In [2]:
import ClassyPrelude
import BiProductionsLib
import qualified Data.Vector as V

: 1

In [3]:
--          "  The quick brown   fox jumped  over   the  lazy dog's  back ."
--wordId     00000 00001 00010 00011  00100 00101 00110 00111 01000 01001
--modifierId 00011 00011 00011 00011  01001 00100 01001 01000 01001 00011
--analogy    Analogy 0 00011 01001 00100

s1OMList =  [    3,   3,     3,    3,     9,    4,    9,    8,    9,    3] :: [Int]
s1ModifiersList = map (\(x,y) -> Modifier x y) $ zip ([0..] :: [Int]) s1OMList
s1Tenses = V.singleton (NTPast, NTPlusDelta)
s1Modifiers = V.fromList s1ModifiersList
s1Analogies = V.singleton (Analogy 0 3 9 4)
s1 = Sentence s1Tenses s1Modifiers s1Analogies
--bits = fmap V.fromList $ encodeOneHotOutput defaultOneHotCaps s1
--encodeOneHotOutput defaultOneHotCaps s1
--fmap (getBitGroups defaultOneHotCaps) bits
s1

Sentence [(NTPast,NTPlusDelta)] [Modifier 0 3,Modifier 1 3,Modifier 2 3,Modifier 3 3,Modifier 4 9,Modifier 5 4,Modifier 6 9,Modifier 7 8,Modifier 8 9,Modifier 9 3] [Analogy 0 3 9 4]


: 1

In [4]:
--          " Jack   ate   the twinkie     ;  then    he   got  sick     ."
--wordId     00000 00001 00010   00011 00100 00101 00110 00111 01000 01001
--modifierId 00000 00011 00011   00000     - 01000 00110 01000 00110     -
--analogy    Analogy 0 00000 00011 00001, Analogy 1 00110 01000 00111 

s2OMList =  [    0,   3,     3,    0,    -1,    8,    6,    8,    6,    -1] :: [Int]
s2ModifiersList = map (\(x,y) -> Modifier x y) $ zip ([0..] :: [Int]) s2OMList
f2ModifiersList = filter (\(Modifier _ y) -> y >= 0) s2ModifiersList
s2Tenses = V.fromList [(NTPast, NTPlusDelta), (NTPlusDelta, NTPlusDelta)]
s2Modifiers = V.fromList f2ModifiersList
s2Analogies = V.fromList [(Analogy 0 0 3 1), (Analogy 1 6 8 7)]
s2 = Sentence s2Tenses s2Modifiers s2Analogies
--bits = fmap V.fromList $ encodeOneHotOutput defaultOneHotCaps s1
--encodeOneHotOutput defaultOneHotCaps s1
--fmap (getBitGroups defaultOneHotCaps) bits
s2

Sentence [(NTPast,NTPlusDelta),(NTPlusDelta,NTPlusDelta)] [Modifier 0 0,Modifier 1 3,Modifier 2 3,Modifier 3 0,Modifier 5 8,Modifier 6 6,Modifier 7 8,Modifier 8 6] [Analogy 0 0 3 1,Analogy 1 6 8 7]


: 1

In [5]:
decodeEncodeOneHotOutput defaultOneHotCaps s1
--bits = fmap V.fromList $ encodeOneHotOutput defaultOneHotCaps s2
--fmap (getBitGroups defaultOneHotCaps) bits
decodeEncodeOneHotOutput defaultOneHotCaps s2

Right (Sentence [(NTPast,NTPlusDelta)] [Modifier 0 3,Modifier 1 3,Modifier 2 3,Modifier 3 3,Modifier 4 9,Modifier 5 4,Modifier 6 9,Modifier 7 8,Modifier 8 9,Modifier 9 3] [Analogy 0 3 9 4],True)
Right (Sentence [(NTPast,NTPlusDelta),(NTPlusDelta,NTPlusDelta)] [Modifier 0 0,Modifier 1 3,Modifier 2 3,Modifier 3 0,Modifier 5 8,Modifier 6 6,Modifier 7 8,Modifier 8 6] [Analogy 0 0 3 1,Analogy 1 6 8 7],True)


: 1

In [5]:
ords = wordOrdinals ["killing", "trashing", "started", "killed", "bleed", "dragoned", "passer", "Blade"]
ords "Killed"
padded = padOrdinals (2 ^ 16) (2 ^ 8) ords
padded "Killed"

([1,0,0],[1,0])

<interactive>:49:23: warning: [-Wtype-defaults]
    • Defaulting the following constraints to type ‘Integer’
        (Integral b0) arising from a use of ‘^’ at <interactive>:49:23-28
        (Num b0) arising from the literal ‘16’ at <interactive>:49:27-28
    • In the first argument of ‘padOrdinals’, namely ‘(2 ^ 16)’
      In the expression: padOrdinals (2 ^ 16) (2 ^ 8) ords
      In an equation for ‘padded’:
          padded = padOrdinals (2 ^ 16) (2 ^ 8) ords

<interactive>:49:32: warning: [-Wtype-defaults]
    • Defaulting the following constraints to type ‘Integer’
        (Integral b0) arising from a use of ‘^’ at <interactive>:49:32-36
        (Num b0) arising from the literal ‘8’ at <interactive>:49:36
    • In the second argument of ‘padOrdinals’, namely ‘(2 ^ 8)’
      In the expression: padOrdinals (2 ^ 16) (2 ^ 8) ords
      In an equation for ‘padded’:
          padded = padOrdinals (2 ^ 16) (2 ^ 8) ords
([0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0]

: 1

In [8]:
-- Since we are going for comprehension, it seems unnecessary to digest sentences longer 
-- than 30 words. See:
-- https://insidegovuk.blog.gov.uk/2014/08/04/sentence-length-why-25-words-is-our-limit/
-- The following is a one-hot encoding of a Sentence given these caps.
--wordCap :: Integer
wordCap = 32*2 -- Doubled for compound sentences

--tensesCap :: Integer
tensesCap = 2 -- Two tenses possible given possibility of compound sentences

encodeCaps = OneHotCaps { ohcWords = wordCap, ohcTenses = tensesCap }

-- The encoding will go like this:
-- Input: (29 + log_2 wordCap)*wordCap = 2176
--   wordCap blocks of:
--     1 bit: is present
--     1 bit: is capitalized
--     1 bit: is time
--     16 bits: most significant bits of word ordinal sorted forward
--     8 bits: most significant bits of word ordinal sorted reverse
--     log_2 wordCap bits: word id
-- Output: tensesCap*11 + wordCap*(5 + log_2 tensesCap + Log_2 wordCap) = 726
--   tensesCap blocks of:
--     1 bit: is present
--     9 bit: select AbstractTime start
--     9 bit: select AbstractTime end
--   wordCap blocks of:
--     1 bit: is present
--     1 bit: is modifier
--     1 bit: is subject
--     1 bit: is object
--     1 bit: is analogy
--     log_2 tensesCap bits: select tense
--     log_2 wordCap bits: referent word id
--       Notes: For modifier, referent word id refers to object or analogy that is being modified
--              For subject, referent word id refers to subject
--              For object, referent word id refers to subject
--              For analogy, referent word id refers to object

--encodeOneHot :: OneHotCaps -> Dictionary -> Sentence -> Either Text ([Double], [Double])
encodeOneHot caps dictionary (Sentence input tenses objects analogies) = 


<interactive>:100:24: warning: [-Wunused-matches]
    Defined but not used: ‘inSent’

<interactive>:100:31: warning: [-Wunused-matches]
    Defined but not used: ‘tenses’

<interactive>:100:38: warning: [-Wunused-matches]
    Defined but not used: ‘objects’

<interactive>:100:46: warning: [-Wunused-matches]
    Defined but not used: ‘analogies’

<interactive>:100:59: warning: [-Wdeprecations]
    In the use of ‘undefined’ (imported from ClassyPrelude):
    Deprecated: "It is highly recommended that you either avoid partial functions or provide meaningful error messages"


: 1